In [1]:
from urllib.parse import urlparse
from urllib.parse import parse_qs
import math
from youtube_transcript_api import YouTubeTranscriptApi

In [2]:
# yt_video_list=["https://www.youtube.com/watch?v=4qh_rguqdD4",
#                "https://www.youtube.com/watch?v=rR_mlBb-5-k&t=1211s"
#                ,"https://www.youtube.com/watch?v=k-4kdiEqCYg",
#                "https://www.youtube.com/watch?v=GX4nd41Iu-Q",
#                "https://www.youtube.com/watch?v=L65CxuCt-bg",
#                "https://www.youtube.com/watch?v=hSXlv26f_Kg&t=783s",
#                "https://www.youtube.com/watch?v=BYkPDg7RZa4",
#                "https://www.youtube.com/watch?v=_2k1pHKSHtg",
#                "https://www.youtube.com/watch?v=ibQn5wau_VA",
#                "https://www.youtube.com/watch?v=Jie6URfi1HY"]
yt_video_list=["https://www.youtube.com/watch?v=4qh_rguqdD4",
               "https://www.youtube.com/watch?v=rR_mlBb-5-k&t=1211s"]

In [3]:
list_of_chunks_all_videos=[]    
for video_url in yt_video_list:
    list_of_lines=[]  #for current video
    url_data=urlparse(video_url)
    query = parse_qs(url_data.query)
    video_id = query["v"][0]
    print(video_id)
    transcritps = YouTubeTranscriptApi.get_transcript(video_id)
    for i in transcritps:
        time=i['start']
        time=math.floor(time)
        hours=int(time//3600)
        minutes=int((time%3600)//60)
        secs=time%60
        # print( f"{hours:02}:{minutes:02}:{secs:06.3f}")
        i['start']=f"{hours:02}:{minutes:02}:{secs:06.3f}"
        ## and adding youtube video
        i['video_url']=video_url   # adding youtube video url
        list_of_lines.append({'time':i['start'],'text':i['text'],'video_url':i['video_url']})
    list_of_chunks=[]
    chunk=[]
    chunk.append(list_of_lines[0])
    chunk.append(list_of_lines[1])
    no=0
    for i in list_of_lines[2:]:
        chunk.append(i)
        no+=1
        if len(chunk)>8:
            chunk.pop(0)
        if (no)%6==0:
            chunk_start=chunk[0]['time']
            chunk_url=chunk[0]['video_url']
            chunk_text=''
            count=0
            for j in chunk:
                count+=1
                chunk_text+=j['text']
                chunk_text+=' '
            # print(count)
            # print(chunk_text)
            # print()
            list_of_chunks.append({'time':chunk_start,'text':chunk_text,'video_url':video_url})
            no=0
    if len(chunk)!=0:
        chunk_start=chunk[0]['time']
        chunk_url=chunk[0]['video_url']
        chunk_text=''
        count=0
        for j in chunk:
            count+=1
            chunk_text+=j['text']
            chunk_text+=' '
        # print(count)
        # print(chunk_text)
        # print()
        list_of_chunks.append({'time':chunk_start,'text':chunk_text,'video_url':video_url})
    no=0
    print(len(list_of_chunks))
    list_of_chunks_all_videos.extend(list_of_chunks)
    print("the len now is ",len(list_of_chunks_all_videos))
    

4qh_rguqdD4
101
the len now is  101
rR_mlBb-5-k
99
the len now is  200


In [4]:
print(list_of_chunks_all_videos)

[{'time': '00:00:00.000', 'text': "e [Music] [Music] hello hello and happy Monday it's a May bank holiday at least in the UK I hope it you're having a wonderful Monday either way wherever you are if you're not having a bank holiday my name is ", 'video_url': 'https://www.youtube.com/watch?v=4qh_rguqdD4'}, {'time': '00:00:50.000', 'text': "either way wherever you are if you're not having a bank holiday my name is Mandy Seline I'm a director and DP and I'm the founder of breadbox Studios which is a boutique post and production house kind of based in the UK we specialize in branded and digital first content in general so we work with a lot ", 'video_url': 'https://www.youtube.com/watch?v=4qh_rguqdD4'}, {'time': '00:01:01.000', 'text': "specialize in branded and digital first content in general so we work with a lot of creators as well for those of you who don't know the last few Mondays of May I've been hosting these challenges every Monday and it's the last Monday of May so this is my la

In [5]:
# put into a df and vectorizing

In [6]:
import pandas as pd

In [7]:
df=pd.DataFrame(list_of_chunks_all_videos)

In [8]:
df.shape

(200, 3)

In [9]:
# df.head(400)

In [10]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings("ignore", message="Unverified HTTPS request")
warnings.filterwarnings("ignore", message="InsecureRequestWarning")

In [11]:
from opensearchpy import OpenSearch

CLUSTER_URL = 'https://localhost:9200'

def get_os_client(cluster_url = CLUSTER_URL,
                  username='admin',
                  password='admin'):
    '''
    Get OpenSearch client
    :param cluster_url: cluster URL like https://ml-te-netwo-1s12ba42br23v-ff1736fa7db98ff2.elb.us-west-2.amazonaws.com:443
    :return: OpenSearch client
    '''
    client = OpenSearch(
        hosts=[cluster_url],
        http_auth=(username, password),
        verify_certs=False
    )
    return client

client = get_os_client()

/Users/vpatwal/Desktop/trying_summarization/.venv/lib/python3.10/site-packages/opensearchpy/connection/http_urllib3.py:208: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [12]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "msmarco-distilbert-base-tas-b"


In [13]:
model = SentenceTransformer(MODEL_NAME)

In [14]:
EMBEDDING_DIM = model.encode(["Sample sentence"])[0].shape[0]
print(EMBEDDING_DIM)#will be used ahead

768


In [15]:
index_name = "new_one2"

index_body = {
  "settings": {
    "index": {
      # "knn": False,
      # "knn.algo_param.ef_search": 100
    }
    ,
    "analysis": {
      "analyzer": {
        "custom_analyzer": {
          "tokenizer": "standard",
          "filter": ["lowercase", "synonym_filter"]
        }
      },
      "filter": {
        "synonym_filter": {
          "type": "synonym",
          "synonyms": [
            "psp, photoshop, psp"
          ]
        }
      }}
  },
  "mappings": { #how do we store, 
    "properties": {
        "embedding": {
          "type": "knn_vector", #we are going to put 
          "dimension": EMBEDDING_DIM,
        #   "method": {
        #     "name": "hnsw",
        #     "space_type": "l2",
        #     "engine": "faiss",
        #     "parameters": {
        #       "ef_construction": 128,
        #       "m": 24
        #     }
        #  }
     },
     "url":{
         "type":"keyword"  #In Elasticsearch, there are several datatypes available for fields, and "keyword" is one of them. It is used for fields that contain structured data like IDs, email addresses, status codes, tags, etc., where exact matching is required.

     }
    }
}}


In [18]:
response = client.indices.delete(index = index_name)

In [19]:
response = client.indices.create(index=index_name, body=index_body)

In [20]:
df.head()
df.columns

Index(['time', 'text', 'video_url'], dtype='object')

In [21]:
import pandas as pd

for idno, row in df.iterrows():
    print(f"Id: {idno}, Start: {row['time']}, Overview: {row['text']},url:{row['video_url']}")
    # original_title = row['time']
    overview = row['text']
    id = row['time']
    st_time=row['time']
    url=row['video_url']
    # Sentence transformer model takes list of documents as input and returns list of embeddings.
    embedding = model.encode([overview])[0]
    # We are inserting a data point with 3 attribute, "id", "text" and "embedding" as knn_vector type.
    my_doc = {"id": str(idno), "plot": overview, "embedding": embedding,"time" : st_time,"url":url}
    res = client.index(
        index=index_name,
        body=my_doc,
        id = str(idno),
        refresh = True
        )

Id: 0, Start: 00:00:00.000, Overview: e [Music] [Music] hello hello and happy Monday it's a May bank holiday at least in the UK I hope it you're having a wonderful Monday either way wherever you are if you're not having a bank holiday my name is ,url:https://www.youtube.com/watch?v=4qh_rguqdD4
Id: 1, Start: 00:00:50.000, Overview: either way wherever you are if you're not having a bank holiday my name is Mandy Seline I'm a director and DP and I'm the founder of breadbox Studios which is a boutique post and production house kind of based in the UK we specialize in branded and digital first content in general so we work with a lot ,url:https://www.youtube.com/watch?v=4qh_rguqdD4
Id: 2, Start: 00:01:01.000, Overview: specialize in branded and digital first content in general so we work with a lot of creators as well for those of you who don't know the last few Mondays of May I've been hosting these challenges every Monday and it's the last Monday of May so this is my last one sad face but

In [22]:
# "query": {
#     "hybrid": {
#         "queries": [
#         {
#             "match": {
#             "passage_text": {
#                 "query": "Hi world"
#             }
#             }
#         },
#         {
#             "neural": {
#             "passage_embedding": {
#                 "query_text": "Hi world",
#                 "model_id": "aVeif4oB5Vm0Tdw8zYO2",
#                 "k": 5
#             }
#             }
#         }
#         ]
#     }
# }

In [31]:
import nltk
from nltk.corpus import wordnet


In [33]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/vpatwal/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [57]:
""" Example query text """
# user_query = "Applying Lens Blur"
# user_query = "hide content"
# user_query = "expanding"
# user_query="deleting people"
# user_query='How does the Generative AI Remove feature in Lightroom work?'
# user_query="What steps does Jacob describe for creating a drop shadow in Illustrator?"
# user_query="Inserting drop shadow"
# user_query="cut it"
# user_query="How to create a montage using markers in Premiere Pro?"
# user_query="select and arrange photos for a video montage?"
# user_query="add markers to a timeline for precise editing"
# user_query="delete unwanted sections of a video sequence in Premiere Pro"
# user_query="Asset Availability and Customization"
# user_query="Asset availability"
user_query="photoshop is good"
# user_query="psp is good"
synonym_groups = [
    [ 'photoshop','psp']
]

def normalize_text(text):
    tokens = nltk.word_tokenize(text)
    normalized_tokens = []

    for token in tokens:
        # Check if the token is in any synonym group
        for group in synonym_groups:
            if token in group:
                normalized_tokens.append(group[0])  # Use the first word in the group as the canonical form
                break
        else:
            normalized_tokens.append(token)

    return ' '.join(normalized_tokens)
# print()


# Example usage with SentenceTransformer
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# sentence = "He was very happy because his pizza was large."
user_query = normalize_text(user_query)
print(user_query)
""" Embedding the query by using the same model """
query_embedding = model.encode((user_query))


query_body = { #most initial one
    "query": {"knn": {"embedding": {"vector": query_embedding, "k": 10}}},
    "_source": False,
    "fields": ["id", "title", "plot","time","url"],
    "size":10  # vivek added this , top 10
}
query_body = {  # works but its kinda post filtering
    #In this scenario, we can say that the URL matching is not really a pre-filtering or post-filtering step. Instead, it's more like a conjunctive filtering step, where both the URL match and KNN search conditions must be true for a document to be considered a match.

# The URL match is not filtering the documents before the KNN search,
#  nor is it filtering the results after the KNN search. Instead,
#    it's a condition that's applied in parallel with the KNN search, 
#    and both conditions must be satisfied for a document to be included in the results.

# So, in this case, we can't really say it's pre-filtering or post-filtering,
#  but rather a conjunctive filtering approach.


    # "from":15,  
    # "size":15,
    "query": {
        "bool": {
            "must": [
                {
                    "match": {
                        "url": {
                            "query": "https://www.youtube.com/watch?v=rR_mlBb-5-k&t=1211s",
                            # "query":"https://www.youtube.com/watch?v=4qh_rguqdD4",
                            # "boost": 1000  # Adjust boost factor for this specific URL match
                        }
                    }
                },
                {
                    "knn": {
                        "embedding": {
                            "vector": query_embedding,
                            "k": 50
                        }
                    }
                }
            ]
        }
    },
    "_source": False,
    "fields": ["id", "title", "plot", "time", "url"],
    "size": 200
}
# query_body={
#   "query": {
#     "bool": {
#       "should": [
#         {
#           "match": {
#             "url": {
#               "query": "https://www.youtube.com/watch?v=4qh_rguqdD4",
#               "boost": 2
#             }
#           }
#         },
#         {
#           "knn": {
#             "embedding": {
#               "vector": query_embedding,
#               "k": 20
#             }
#           }
#         }
#       ],
#       "minimum_should_match": 1
#     }
#   },
#   "_source": False,
#   "fields": ["id", "title", "plot", "time", "url"],
#   "size": 100
# }

# query_body = {
#     "query": {
#         "bool": {
#             "filter": [
#                 {
#                     "match": {
#                         "url": {
#                             # "query": "https://www.youtube.com/watch?v=rR_mlBb-5-k&t=1211s",
#                             "query":"https://www.youtube.com/watch?v=4qh_rguqdD4",
#                             "boost": 1  # Adjust boost factor for this specific URL match
#                         }
#                     }
#                 }
#                 ],
#                 "should":
#                 [
#                     {
#                     "knn": {
#                         "embedding": {
#                             "vector": query_embedding,
#                             "k": 100
#                         }
#                     }
#                     }
#                 ]
#         }
#     },
#     "_source": False,
#     "fields": ["id", "title", "plot", "time", "url"],
#     "size": 10
# }



query_body = {  #  😆 PREFILTERING DONEEEEE!
    "from":1,  
    "size":15,
"query": {
    "script_score": {
      "query": {
        "bool": {
          "filter": {
            "term": {
              "url": "https://www.youtube.com/watch?v=rR_mlBb-5-k&t=1211s"
            }
          }
        }
      },
      "script": {
        "source": "knn_score",
        "lang": "knn",
        "params": {
          "field": "embedding",  # Assuming your k-NN field is named "embedding"
          "query_value": query_embedding,
          "space_type": "l2"
        }
      }
    }
  },
  # "_source": False,
  # "fields": ["id", "title", "plot", "time", "url"],
  # "size": 100
}


results = client.search(
    body=query_body,
    index=index_name
)



## must and should be's are similar but lets say i took k=10, and if we have it , then if it doesnt match it can still be used 
## i think should is more strict, like in must it runs both and then weighs
## eg for k =10 and size =100

photoshop is good


In [58]:
# print(query_embedding)
# type(query_embedding)
# query_embedding.shape

In [59]:
print(results)

{'took': 9, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 99, 'relation': 'eq'}, 'max_score': 0.017001813, 'hits': [{'_index': 'new_one2', '_id': '114', '_score': 0.016855258, '_source': {'id': '114', 'plot': "you can use it's pretty simple uh and there's some really fun use applications for that so typically the first thing I like to do is make sure that my image illustrator illustration I should say is grouped up what I'm going to do as well just so we can see two examples is I'm just going ", 'embedding': [0.3047407269477844, 0.055433787405490875, -0.29349634051322937, -0.041164580732584, 0.11165256798267365, -0.09885764867067337, -0.5238146781921387, -0.16421860456466675, -0.36180567741394043, -0.17345160245895386, 0.04625493660569191, 0.29445546865463257, -0.044193703681230545, 0.01478097029030323, 0.20686008036136627, 0.17405898869037628, -0.15474359691143036, 0.15562182664871216, -0.18846550583839417, 0.21946

In [61]:
# for i, result in enumerate(results["hits"]["hits"]):
#     # print(i,results)
#     plot = result['fields']['plot'][0]
#     url = result['fields']['url'][0]
#     score = result['_score']
#     time=result['fields']['time'][0]
#     id=result["_id"]
#     print(f"{id}., Start Time: {time} Score: {score}, Url: {url}  Plot: {plot}")
for i, result in enumerate(results["hits"]["hits"]):
    # print(i,results)
    plot = result['_source']['plot']
    url = result['_source']['url']
    score = result['_score']
    # time=result['fields']['time'][0]
    # id=result["_id"]
    # print(f"{id}., Start Time: {time} Score: {score}, Url: {url}  Plot: {plot}")
    # print(result['_source'].keys())
    print(score)
    print(url)
    print(plot)


0.016855258
https://www.youtube.com/watch?v=rR_mlBb-5-k&t=1211s
you can use it's pretty simple uh and there's some really fun use applications for that so typically the first thing I like to do is make sure that my image illustrator illustration I should say is grouped up what I'm going to do as well just so we can see two examples is I'm just going 
0.015930425
https://www.youtube.com/watch?v=rR_mlBb-5-k&t=1211s
seeing what my favorite option would be personally I'm really liking this one I wonder if we can get any others that look just as good I like this I also really like this these are not free though so let's scroll back up it's not overom complicated let's go with this guy lots 
0.015813481
https://www.youtube.com/watch?v=rR_mlBb-5-k&t=1211s
notified but of course when you do you'll get notified of all of the illustrator challenges as well as any other challenges that you're interested in and also make sure you head over to let's see if we can there we go to the Discord uh the D

In [51]:
len(results["hits"]['hits'])

15

In [52]:
print(results["hits"]["hits"])

[{'_index': 'new_one2', '_id': '152', '_score': 0.015136108, '_source': {'id': '152', 'plot': "is with black and white right this is with some simple illustrations that you know they look okay but I want to add some color to this so let's do it with some illustrations that have color same concept I went ahead and on my second artboard over here I prepared some characters that I thought were really ", 'embedding': [0.10741880536079407, -0.17107881605625153, -0.16599169373512268, 0.28977036476135254, 0.1899518072605133, -0.1748756319284439, -0.3201911747455597, -0.12507574260234833, -0.3180484175682068, -0.4971829056739807, 0.3794194161891937, 0.23521676659584045, -0.007957401685416698, -0.25628766417503357, 0.21656808257102966, 0.2521136999130249, -0.12943820655345917, 0.031742267310619354, -0.40365639328956604, -0.26147767901420593, -0.06259172409772873, -0.0777277871966362, 0.10573509335517883, -0.16569049656391144, -0.004926054738461971, -0.212284117937088, 0.45791706442832947, -0.32

In [ ]:
# client({from=0&size=10})

In [ ]:
# df.to_csv("table3")